This is Version 2 - In this model, the following changes have been made :
a) Increase the order of eqn for todal evolution - to l~ 6
b) Include the term for dw/ds explicitly instead of assuming synchronization at all times
c) Incldue tidal dissipation in the asteroid.

d) Keep track of the velocity of the object - (r,v_r,v_theta,v_total)


In [49]:
from astropy import constants
from scipy import integrate

G_val = constants.G.si.value

Cp_rock = 1000. # J/kg/K

M_mars = 6.41693*(10**23.) # Mars mass, kg
R_mars = 3389.5*1e3 # Mars, m
rho_mars = 3933 #kg/m^3
V_phobos = 5689*1e9 # +/- 60*1e9 m^3
M_phobos = 1.065*(10**16.) # Phobos, +/- .015 kg
R_phobos = 11.3*1e3  # Phobos, m
rho_phobos = 1860. # +/- 31 kg/m^3
ecc_phobos =  0.01511
a_phobos = 9378*1e3 # Semi-major Axis, m 
w_phobos = 1./27562. # 1/Rotation period of phobos ~ 7hr 39 mins, 1/s

mu_phobos_monolith = 8e4 # Dimensionless, from 8e4 - 8e6
k2_mars = .148  # In Nimmo 2013; .148 +/- .017 ; Other Sets in Jacobsen 2014 - 0.1837 +/- .0009
Q_mars = 88.    # In Nimmo 2013 88. +/- 16, 170+/- 20 ; Other Sets in Jacobsen 2014 - 99.57 +/- 4.9 

mu_phobos_rubble = sqrt(mu_phobos_monolith/1e-2) # Dimensionless, ~ sqrt(mu/epsilon_yield_strain); ~ from 2.85e3 - 2.85e4
k2_phobos = 1.5/(1. + mu_phobos_rubble)
Q_phobos = 100. # Can be between 50 - 100

mu_phobos_C1 = (4.*pi/19.)*(rho_phobos**2.)*(R_phobos**2.)*G_val/k2_phobos
const_Ecc1 = -(57./8.)*(k2_mars/Q_mars)*sqrt(G_val/M_mars)*(R_mars**5.)*M_phobos
const_Ecc2 = -(21./2.)*(k2_phobos/Q_phobos)*sqrt(G_val*M_mars)*(R_phobos**5.)*(M_mars/M_phobos)
const_E1 = (21./2.)*(k2_phobos/Q_phobos)*sqrt(G_val*M_mars)*(R_phobos**5.)*(G_val*M_mars*M_mars)

kappa = (rho_phobos/rho_mars)*(R_phobos/R_mars)**3.
rad_ratio = R_phobos/R_mars
const_w_s1 = (8./19.)*(5./2.)*((pi*G_val*R_mars)**2.)*(rho_mars**3.)*(R_mars/R_phobos)*kappa*(R_mars**6.)/Q_phobos/mu_phobos_C1
#const_w_s1 = 5.*pi*(k2_phobos/Q_phobos)*G_val*rho_mars*(rho_mars/rho_phobos)
    
mu_mars_C1 = (4.*pi/19.)*(rho_mars**2.)*(R_mars**2.)*G_val/k2_mars
const_a1 = (8.*sqrt(3)/19.)*(pi*G_val)**1.5*(R_mars**2.)*(rho_mars**2.5)*kappa*sqrt(1.+kappa)*(R_mars**5.5)/mu_mars_C1/Q_mars
    
def dX_dt(t,X):
    """ X is [a,e,w_s,E] - for moon/Phobos , E is the Energy, a is in units of R_mars """
    dt_e = (const_Ecc1 + const_Ecc2)*(X[1])/((X[0]*R_mars)**6.5)
    dt_E = const_E1*(X[1]*X[1])/((X[0]*R_mars)**7.5)
    w_t1 = + (19./22.)*(rad_ratio**2.)/(X[0]**2.)
    w_t2 = + (380./459.)*(rad_ratio**4.)/(X[0]**4.)
    w_t3 = + (475./584.)*(rad_ratio**6.)/(X[0]**6.)
    w_t4 = + (133./165.)*(rad_ratio**8.)/(X[0]**8.)
    a_t1 = + (19./22.)/(X[0]**2.)
    a_t2 = + (380./459.)/(X[0]**4.)
    a_t3 = + (475./584.)/(X[0]**6.)
    a_t4 = + (133./165.)/(X[0]**8.)
    n_val = 1e10*sqrt(G_val*M_mars/(X[0]*R_mars)**3.)/2./pi # Since 1/w_s = 2*pi/n
    #print sign(n_val-X[2])
    dt_w = sign(n_val-X[2])*(const_w_s1/(X[0]**6.))*(1. + w_t1 + w_t2 + w_t3 + w_t4)
    dt_a = -1.*(const_a1/((X[0]*R_mars)**5.5))*(1. + 51.*X[1]*X[1]/4.)*(1. + a_t1 + a_t2 + a_t3 + a_t4) # Is really dt_a/R_mars
    if (X[1] <=0.):
        dt_e =0.
    if (X[0] <=1.):
        dt_a =0.
        dt_e =0.
        dt_w =0.
        dt_E =0.
    return array([dt_a,dt_e,dt_w*0.,dt_E])
max_t =80.*1e6*3.154e7                            # unit is per seconds
dt = 1e4*3.154e7
X_init = array([a_phobos/R_mars,ecc_phobos,1e10*w_phobos,0.])               # initials conditions: [a,e]
abc = integrate.ode(dX_dt).set_integrator('vode',nsteps=1e8)
abc.set_initial_value(X_init,0.0)

t_vX=[]
a_intX=[]
e_intX=[]
w_s_intX=[]
E_intX=[]

a_intX.append(X_init[0])
e_intX.append(X_init[1])
w_s_intX.append(X_init[2])
E_intX.append(X_init[3])
t_vX.append(0.)
while abc.t < max_t :
	#print abc.t/max_t
	X = abc.integrate(abc.t+dt)
	a_intX.append(X[0])
	e_intX.append(X[1])
	w_s_intX.append(X[2])
	E_intX.append(X[3])
	t_vX.append(abc.t)
	#print abc.t/max_t
    
a_int = array(a_intX)
e_int = array(e_intX)
#w_s_int = array(w_s_intX)
E_int = array(E_intX)
t_v = array(t_vX)

r_peri =a_int*(1. - e_int) # periapse location 
r_apo = a_int*(1. + e_int) # periapse location 
Vel_r_max = sqrt(G_val*M_mars*(2./r_apo- 1./a_int)/R_mars)# Total Velocity at the periapse for the given a, m/s
Vel_r_min = sqrt(G_val*M_mars*(2./r_peri - 1./a_int)/R_mars)# Total Velocity at the periapse for the given a, m/s
#semilogx(t_v[a_int>1.]/3.154e7/1e6,E_int[a_int>1.]/M_phobos/Cp_rock,'g-')
#n_val = 1e5*sqrt(G_val*M_mars/a_int**3./R_mars**3.)/2./pi

a_int_orig = a_int.copy()
e_int_orig = e_int.copy()


In [50]:
% pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [51]:
M_mars = 6.41693*(10**23.) # Mars mass, kg
R_mars = 3389.5*1e3 # Mars, m
rho_mars = 3933 #kg/m^3
V_phobos = 5689*1e9 # +/- 60*1e9 m^3
M_phobos = 1.065*(10**16.) # Phobos, +/- .015 kg
R_phobos = 11.3*1e3  # Phobos, m
rho_phobos = 1860. # +/- 31 kg/m^3
ecc_phobos =  0.01511
a_phobos = 9378*1e3 # Semi-major Axis, m 
w_phobos = 1./27562. # 1/Rotation period of phobos ~ 7hr 39 mins, 1/s

Shape_phobos_a = 13.03*1e3 
Shape_phobos_b = 11.40*1e3 
Shape_phobos_c = 9.14*1e3 
Shape_phobos_alpha = Shape_phobos_c/Shape_phobos_a
Shape_phobos_beta = Shape_phobos_b/Shape_phobos_a

Ax_phobos = Shape_phobos_alpha*Shape_phobos_beta*.880592
Ay_phobos = Shape_phobos_alpha*Shape_phobos_beta*1.03759
Az_phobos = Shape_phobos_alpha*Shape_phobos_beta*1.35361
w_centr = sqrt(G_val*M_mars/(a_int*R_mars)**3.)

Term1 = rho_phobos*w_centr*w_centr + (8.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3. -2.*pi*G_val*Ax_phobos*rho_phobos**2.
Term2 = rho_phobos*w_centr*w_centr - (4.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3. -2.*pi*G_val*Ay_phobos*rho_phobos**2.
Term3 =  - (4.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3. -2.*pi*G_val*Az_phobos*rho_phobos**2.
sigma_x = Term1*Shape_phobos_a**2./5.
sigma_y = Term2*Shape_phobos_b**2./5.
sigma_z = Term3*Shape_phobos_c**2./5.


In [52]:
## Hoek Model ... 
D = 1. 
mi = 5. 
GSI = 5. 
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))
Y_0 = 100*1e6 # Pascal (10-100 Mpa) # sigma_ci

rhs1_H = Y_0*(-mb*(sigma_z)/Y_0 + s_hoek)**a_hoek - sigma_z
#lhs2_H = Y_0*mb*(Term2/Y_0 + s_hoek)**a_hoek + Term2

plot(a_int[a_int>1.+160./R_mars],(sigma_x[a_int>1.+160./R_mars]),'-',color='r',linewidth = .7)
#plot(a_int[a_int>1.+160./R_mars],(sigma_x[a_int>1.+160./R_mars]),'-',color='r',linewidth = .7)
plot(a_int[a_int>1.+160./R_mars],rhs1_H[a_int>1.+160./R_mars],'o-',color='b',linewidth = .7)



In [66]:
## Hoek Model ... 
D = 1. 
mi = 10.
GSI = 15.
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))
Y_0 = 1e6*25.

Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
sigma_cm = Y_0*Term_hoek
print,sigma_cm/1e6

(<function print>, 0.30443139858408536)

In [35]:
friction_ang = 45.*pi/180.
.25e6*(1. - sin(friction_ang))/2./cos(friction_ang)/1e6

0.051776695296636886

In [59]:

sigma_x = Term1*Shape_phobos_a**2./5.
sigma_y = Term2*Shape_phobos_b**2./5.
sigma_z = Term3*Shape_phobos_c**2./5.
J2_term = sqrt((sigma_x-sigma_y)**2. + (sigma_y-sigma_z)**2. + (sigma_z-sigma_x)**2. )/sqrt(6.)

r_mean_particle_size = 1e-6 # 1 micron 
Y_minimum = 1.56e-4/r_mean_particle_size # ~ 100 Pascal

import numpy as np
import pylab as plt

fig_width_pt = 2.*130.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height = 1.3*fig_width*golden_mean      # height in inches
params = {'axes.labelsize': 10,
          'text.fontsize': 8,
          'text.fontweight': 30,
          'legend.fontsize': 5,
          'xtick.labelsize': 6.5,
          'axes.linewidth': .3,
          'ytick.labelsize': 6.5}
pylab.rcParams.update(params)
#fig_size =  [fig_width,fig_height]
import matplotlib.gridspec as gridspec

fig,axes = plt.subplots(figsize=(fig_width,fig_height),dpi=250)
c=['r','b','k','g','c','y']

friction_ang = (65.)*pi/180.
s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
I_term = s*(sigma_x + sigma_y + sigma_z)
Cohesion_needed_term = J2_term + I_term
k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
term1 = Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term
term1[term1 < 0] = min(term1[term1 > 0])
axes.semilogy(a_int[a_int>1.+160./R_mars],term1,'-',color='b',linewidth = .7)

friction_ang = (30.)*pi/180.
s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
I_term = s*(sigma_x + sigma_y + sigma_z)
Cohesion_needed_term = J2_term + I_term
k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))

term2 = Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term
term2[term2 < 0] = min(term2[term2 > 0])

x_value = a_int[a_int>1.+160./R_mars]
axes.semilogy(a_int[a_int>1.+160./R_mars],Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term,'-',color='b',linewidth = .7)
axes.fill_between(x_value,term1,term2,facecolor='blue', alpha=0.25)

axes.set_xlim([1.0674454227893777,2.7667797610267])
axes.set_ylabel('Critical Yield\n Strength(MPa)',fontsize=10,color='blue')
axes.set_xlabel('a/R$_{Mars}$',fontsize=10,labelpad=3.5)
#legend([r'$\theta_{f}=15^{\circ}$',r'$\theta_{f}=30^{\circ}$',r'$\theta_{f}=45^{\circ}$',
#        r'$\theta_{f}=60^{\circ}$'],loc='best', framealpha=.75,ncol=2)

## Hoek Model ... 
D = 1. 
mi = 5. 
GSI = 5. 
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))

Y_strength_model = zeros([4,1])
for i in range(0,4):
    friction_ang = (15. + i*15.)*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    
    # Stress models :
    # A ) Damage fraction (Jutzi et al. 2014)
    Y_0 = 100*1e6 # Pascal (10-100 Mpa)
    Y_M = 3.5e9
    #u_i = tan(friction_ang)
    #u_d = tan(friction_ang) # or .55, .8
    #g_phobos = G_val*(M_phobos)/R_phobos**2.
    #Press = rho_phobos*R_phobos*g_phobos/5. 
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    #Y_i = (Y_0 + u_i*Press/(1.+(u_i*Press)/(Y_M - Y_0)))
    #semilogy(a_int[a_int>1.+160./R_mars],Cohesion_needed_term[a_int>1.+160./R_mars]*0.0 + (Y_minimum+ Y_damaged)/1e6,'-.',color=c[i],linewidth = .7)
    #tmp1 = argmin(abs(Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term - Y_damaged/1e6))
    Y_strength_model[i] = (Y_minimum+ Y_damaged)/1e6
    print(friction_ang/(pi/180.),(Y_minimum+ Y_damaged)/1e6)
    
Y_strength_model_Hoek1 = Y_strength_model.copy()


D = 1. 
mi = 15. # 20.
GSI = 25. # 30.
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))
Y_strength_model = zeros([4,1])
for i in range(0,4):
    friction_ang = (15. + i*15.)*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    
    # Stress models :
    # A ) Damage fraction (Jutzi et al. 2014)
    Y_0 = 100*1e6 # Pascal (10-100 Mpa)
    Y_M = 3.5e9
    u_i = tan(friction_ang)
    u_d = tan(friction_ang) # or .55, .8
    g_phobos = G_val*(M_phobos)/R_phobos**2.
    Press = rho_phobos*R_phobos*g_phobos/5. 
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    Y_i = (Y_0 + u_i*Press/(1.+(u_i*Press)/(Y_M - Y_0)))
    #semilogy(a_int[a_int>1.+160./R_mars],Cohesion_needed_term[a_int>1.+160./R_mars]*0.0 + (Y_minimum+ Y_damaged)/1e6,'-.',color=c[i],linewidth = .7)
    tmp1 = argmin(abs(Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term - Y_damaged/1e6))
    Y_strength_model[i] = (Y_minimum+ Y_damaged)/1e6
    print(friction_ang/(pi/180.),(Y_minimum+ Y_damaged)/1e6)

from matplotlib.patches import Ellipse, Polygon
p2 = axes.add_patch(Polygon([[1.0674454227893777,Y_strength_model[0]],[1.0674454227893777,Y_strength_model_Hoek1[3]],
                             [2.7667797610267,Y_strength_model_Hoek1[3]],[2.7667797610267,Y_strength_model[0]]], closed=True,
                      fill=False, hatch='/',linewidth=.5,color='black'))

print(Y_strength_model[0],Y_strength_model_Hoek1[3])


plt.gcf().subplots_adjust(bottom=0.15)
plt.gcf().subplots_adjust(left=.15)
# Stress Model 2 : Holsapple 2008
axes.semilogy(a_int[a_int>1.+160./R_mars],a_int[a_int>1.+160./R_mars]*0. + (Y_minimum)/1e6 + .025*rho_phobos**2./sqrt(R_phobos)/1e6,'--',color='k',linewidth=1.15)

# Stress Model 2 : Goldriech 2009
#mu_phobos_rubble = 2.85e3 # to - 2.85e4
#Y_rigidity = mu_phobos_rubble*(2./19.)*g_phobos*R_phobos*rho_phobos/1e2
#semilogy(a_int[a_int>1.+160./R_mars],Cohesion_needed_term[a_int>1.+160./R_mars]*0.0 + (Y_minimum + Y_rigidity)/1e6,'-',color='b',linewidth=1.5)

# Stress Model 3 : Solid Rock
axes.semilogy(a_int[a_int>1.+160./R_mars],Cohesion_needed_term[a_int>1.+160./R_mars]*0.0 + (Y_minimum + Y_i)/1e6,'-',color='k',linewidth=1.5)

#mu_phobos_rubble = 2.85e4 # to - 2.85e4
#Y_rigidity = mu_phobos_rubble*(2./19.)*g_phobos*R_phobos*rho_phobos/1e2
#semilogy(a_int[a_int>1.+160./R_mars],Cohesion_needed_term[a_int>1.+160./R_mars]*0.0 + (Y_minimum + Y_rigidity)/1e6,'o',color='b')
p1 = axes.add_patch(Polygon([[-1,.1],[-1.3,.2],[-1.1,.3],[-2,.3]], closed=True,
                      fill=True,linewidth=.5,color='blue',alpha=.25))

legend([p1],[r'$\theta_{f}=\,15^{\circ} -\, 60^{\circ} $',],loc='best', framealpha=.75)

Y_strength_model2 = zeros([4,1])

for i in range(0,4):
    friction_ang = (15. + i*15.)*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    
    # Stress models :
    # A ) Damage fraction (Jutzi et al. 2014)
    Y_0 = 100*1e6 # Pascal (10-100 Mpa)
    Y_M = 3.5e9
    u_i = tan(friction_ang)
    u_d = tan(friction_ang) # or .55, .8
    g_phobos = G_val*(M_phobos)/R_phobos**2.
    Press = rho_phobos*R_phobos*g_phobos/5. 
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    Y_i = (Y_0 + u_i*Press/(1.+(u_i*Press)/(Y_M - Y_0)))
    Y_total = (.7*Y_i) + .3*Y_damaged
    #semilogy(a_int[a_int>1.+160./R_mars],Cohesion_needed_term[a_int>1.+160./R_mars]*0.0 + (Y_minimum+ Y_damaged)/1e6,'-.',color=c[i],linewidth = .7)
    Y_strength_model2[i] = (Y_minimum)/1e6
p3 = axes.add_patch(Polygon([[1.0674454227893777,Y_strength_model2[3]],[1.0674454227893777,Y_strength_model2[0]*.95],[2.7667797610267,Y_strength_model2[0]*.95],[2.7667797610267,Y_strength_model2[3]]], closed=True,
                      fill=False, hatch='x',linewidth=.25))

axes.annotate('$Y_{strength}$ - Cohesion', [1.57,Y_strength_model2[3]],
                    xycoords='data',xytext=(75.,3), textcoords='offset points',
                    fontsize=6,
                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=3,
                                        connectionstyle="arc3,rad=0.165",color='k',alpha=0))

#annotate('a', [1.8,.011],
#                    xycoords='data',xytext=(27.,20.), textcoords='offset points',
#                    fontsize=6,color='white',
#                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=5,
#                                        connectionstyle="arc3,rad=0.15",color='k',alpha=1))

#annotate('u', [2.16,.0048],
#                    xycoords='data',xytext=(-10.,30.), textcoords='offset points',
#                    fontsize=6,color='white',
#                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=5,
#                                        connectionstyle="arc3,rad=0.15",color='k',alpha=1))

annotate('$Y_{strength}$ - Hoek model', [1.8,.042],
                    xycoords='data',xytext=(15.,-10.), textcoords='offset points',
                    fontsize=6,
                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=5,
                                        connectionstyle="arc3,rad=0.15",color='k',alpha=0))

#annotate('$Y_{strength}$ - Goldreich & Sari (2009) model', [1.34,.36],
#                    xycoords='data',xytext=(25.,5.), textcoords='offset points',
#                    fontsize=6,
#                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=5,
#                                        connectionstyle="arc3,rad=0.15",color='k',alpha=.7))


k_term_holsapple_model = (0.025*rho_phobos**2./sqrt(R_phobos))
k_term_out = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))

annotate('Minimum $Y_{strength}$ model', [1.57,k_term_holsapple_model/k_term_out/1e6],
                    xycoords='data',xytext=(-50.,-6), textcoords='offset points',
                    fontsize=6,
                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=1,
                                        connectionstyle="arc3,rad=-0.01",color='k',alpha=0))

#legend([p1,p2,p3],[r'Critical Strength : $\theta_{f}=\,15^{\circ} -\, 60^{\circ} $',
#                   r'Fully damaged Model : $\theta_{f}=\,15^{\circ} -\, 60^{\circ} $',
#                    r'30 $\%$ damaged Model : $\theta_{f}=\,15^{\circ} -\, 60^{\circ} $',],loc='best', framealpha=.75)

annotate('$Y_{strength}$ - No damage model', [1.57,10],
                    xycoords='data',xytext=(-50.,3.), textcoords='offset points',
                    fontsize=6,
                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=3,
                                        connectionstyle="arc3,rad=0.165",color='k',alpha=0))
ylim([1e-4,100])

axR = axes.twinx()
axR.semilogy(a_int[a_int>1.+160./R_mars],term1,'-',color='b',linewidth = 0)
#axR.yaxis.tick_right()
#axR.yaxis.set_label_position("right")
axR.set_ylabel('Model Yield\n Strength(MPa)',fontsize=10,color='k')
axR.set_xlim([1.0674454227893777,2.7667797610267])
axR.set_ylim([1e-4,100])
#plt.gcf().subplots_adjust(left=.2)
#plt.gcf().subplots_adjust(right=-1.5)

axes.tick_params(axis='y', colors='blue')
xlim([1.0,2.6])
plot([1.067,1.067],[1e-4,100],'k-')

14.999999999999998 0.118627811215
29.999999999999996 0.0892962663632
45.0 0.0641086977843
59.99999999999999 0.041526141593
14.999999999999998 1.08860061109
29.999999999999996 0.819120794743
45.0 0.587713230319
59.99999999999999 0.380238883981
[ 1.08860061] [ 0.04152614]


/usr/lib/python3/dist-packages/matplotlib/__init__.py:857: UserWarning: text.fontweight is deprecated and replaced with font.weight; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/usr/lib/python3/dist-packages/matplotlib/__init__.py:857: UserWarning: text.fontsize is deprecated and replaced with font.size; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [5]:
savefig("Fig2a_new2.pdf",dpi=1200,bbox_inches='tight')

#py.plot(arr[:,0], arr[:,1], 'o', alpha=0.1, rasterized=True)
#py.savefig('dots.pdf', dpi=400)


In [58]:
k_term_holsapple_model/1e6/k_term

0.0010024328567438451

In [9]:
M_mars = 6.41693*(10**23.) # Mars mass, kg
R_mars = 3389.5*1e3 # Mars, m
rho_mars = 3933 #kg/m^3
V_phobos = 5689*1e9 # +/- 60*1e9 m^3
M_phobos = 1.065*(10**16.) # Phobos, +/- .015 kg
R_phobos = 11.3*1e3  # Phobos, m
rho_phobos = 1860. # +/- 31 kg/m^3
ecc_phobos =  0.01511
a_phobos = 9378*1e3 # Semi-major Axis, m 
w_phobos = 1./27562. # 1/Rotation period of phobos ~ 7hr 39 mins, 1/s

Shape_phobos_a = 13.03*1e3 
Shape_phobos_b = 11.40*1e3 
Shape_phobos_c = 9.14*1e3 
Shape_phobos_alpha = Shape_phobos_c/Shape_phobos_a
Shape_phobos_beta = Shape_phobos_b/Shape_phobos_a

Ax_phobos = Shape_phobos_alpha*Shape_phobos_beta*.880592
Ay_phobos = Shape_phobos_alpha*Shape_phobos_beta*1.03759
Az_phobos = Shape_phobos_alpha*Shape_phobos_beta*1.35361
w_centr = sqrt(G_val*M_mars/(a_int*R_mars)**3.)

Term1 = rho_phobos*w_centr*w_centr + (8.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3. -2.*pi*G_val*Ax_phobos*rho_phobos**2.
Term2 = rho_phobos*w_centr*w_centr - (4.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3. -2.*pi*G_val*Ay_phobos*rho_phobos**2.
Term3 =  - (4.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3. -2.*pi*G_val*Az_phobos*rho_phobos**2.

sigma_x = Term1*Shape_phobos_a**2./5.
sigma_y = Term2*Shape_phobos_b**2./5.
sigma_z = Term3*Shape_phobos_c**2./5.
J2_term = sqrt((sigma_x-sigma_y)**2. + (sigma_y-sigma_z)**2. + (sigma_z-sigma_x)**2. )/sqrt(6.)

r_mean_particle_size = 1e-6 # 1 micron 
Y_minimum = 1.56e-4/r_mean_particle_size # ~ 100 Pascal

import numpy as np
import pylab as plt

fig_width_pt = 2.*130.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height = 1.3*fig_width*golden_mean      # height in inches
params = {'axes.labelsize': 10,
          'text.fontsize': 8,
          'text.fontweight': 30,
          'legend.fontsize': 5,
          'xtick.labelsize': 7.5,
          'axes.linewidth': .3,
          'ytick.labelsize': 7.5}
pylab.rcParams.update(params)
#fig_size =  [fig_width,fig_height]
import matplotlib.gridspec as gridspec

fig, axes = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True,figsize=(fig_width,fig_height),dpi=250)
clf()

fric_ang_all = linspace(5,80,1000)
all_fric_angl = zeros(1000)
loc_fric_angl = zeros(1000)

D = 1. 
mi = 5. # 20.
GSI = 5. # 30.
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))

for i in range(0,1000):
    friction_ang = fric_ang_all[i]*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    
    # Stress models :
    # A ) Damage fraction (Jutzi et al. 2014)
    Y_0 = 10*1e6 # Pascal (10-100 Mpa)
    Y_M = 3.5e9
    u_i = tan(friction_ang)
    u_d = tan(friction_ang) # or .55, .8
    g_phobos = G_val*(M_phobos)/R_phobos**2.
    Press = rho_phobos*R_phobos*g_phobos/5. 
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    Y_i = (Y_0 + u_i*Press/(1.+(u_i*Press)/(Y_M - Y_0)))
    #semilogy(a_int[a_int>1.+160./R_mars],Cohesion_needed_term[a_int>1.+160./R_mars]*0.0 + (Y_minimum+ Y_damaged)/1e6,'-.',color=c[i],linewidth = .7)
    tmp1 = argmin(abs(Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term - Y_damaged/1e6))
    all_fric_angl[i] = friction_ang/(pi/180.)
    loc_fric_angl[i] = a_int[tmp1]

#plot(loc_fric_angl,all_fric_angl,'k-',linewidth=1.1)
loc_fric_angl_S1  = loc_fric_angl.copy()
all_fric_angl_S1 = all_fric_angl.copy()


D = 1. 
mi = 15. # 20.
GSI = 25. # 30.
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))

for i in range(0,1000):
    friction_ang = fric_ang_all[i]*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    
    # Stress models :
    # A ) Damage fraction (Jutzi et al. 2014)
    Y_0 = 10*1e6 # Pascal (10-100 Mpa)
    Y_M = 3.5e9
    u_i = tan(friction_ang)
    u_d = tan(friction_ang) # or .55, .8
    g_phobos = G_val*(M_phobos)/R_phobos**2.
    Press = rho_phobos*R_phobos*g_phobos/5. 
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    Y_i = (Y_0 + u_i*Press/(1.+(u_i*Press)/(Y_M - Y_0)))
    #semilogy(a_int[a_int>1.+160./R_mars],Cohesion_needed_term[a_int>1.+160./R_mars]*0.0 + (Y_minimum+ Y_damaged)/1e6,'-.',color=c[i],linewidth = .7)
    tmp1 = argmin(abs(Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term - Y_damaged/1e6))
    all_fric_angl[i] = friction_ang/(pi/180.)
    loc_fric_angl[i] = a_int[tmp1]

#plot(loc_fric_angl,all_fric_angl,'k-',linewidth=1.1)

from scipy.interpolate import interp1d
f2 = interp1d(all_fric_angl,loc_fric_angl)
all_fric_angl_new = np.linspace(5., 20., 90)
#plot(f2(all_fric_angl_new),all_fric_angl_new,'k-',linewidth=1.1)

from scipy import interpolate
tck = interpolate.splrep(all_fric_angl,loc_fric_angl, s=.05)
all_fric_angl_new2 = np.linspace(20., 80, 90)
ynew = interpolate.splev(all_fric_angl_new2, tck, der=0)
#plot(ynew,all_fric_angl_new2,'k-',linewidth=1.1)


loc_fric_angl_S2  =  hstack([f2(all_fric_angl_new),ynew])
all_fric_angl_S2 = hstack([all_fric_angl_new,all_fric_angl_new2])

f3 = interp1d(all_fric_angl_S2,loc_fric_angl_S2)
loc_fric_angl_S3  =  f3(all_fric_angl_S1)

plot(loc_fric_angl_S1, all_fric_angl_S1, loc_fric_angl_S2, all_fric_angl_S2,'k-',linewidth=1.1,color='black' )
ax=gca()
ax.fill_betweenx(all_fric_angl_S1, loc_fric_angl_S1, loc_fric_angl_S3, where=loc_fric_angl_S1>=loc_fric_angl_S3, hatch='/',facecolor='white',alpha=.9)


title('Hoek Model - Disruption Location',fontsize=8)
ylabel(r'$\theta_{friction} \, (deg)$',fontsize=10,labelpad=3.5)
xlabel('a$_{disruption}$/R$_{Mars}$ ',fontsize=10)


plt.gcf().subplots_adjust(bottom=0.15)
plt.gcf().subplots_adjust(left=.2)
ylim([10,80])
xlim([1.0,2.6])

from matplotlib.patches import Rectangle
ax = gca()
ax.add_patch(Rectangle(( 1.,25.), 1.6,20.,alpha=.7,facecolor='grey',linewidth=.5))
plt.show()


In [10]:
annotate('Likely range \n      for'+r' $\theta_{f}$', [2.14,36.1,],
                    xycoords='data',xytext=(-20,27.), textcoords='offset points',
                    fontsize=9,
                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=.6,
                                        connectionstyle="arc3,rad=0.1",color='k',alpha=1.))



In [11]:
savefig("Fig2b_New2_B.pdf",dpi=900)

#py.plot(arr[:,0], arr[:,1], 'o', alpha=0.1, rasterized=True)
#py.savefig('dots.pdf', dpi=400)
